In [58]:
import open3d as o3d

import copy
import numpy as np
import os
import pathlib
import sys
import time
import inspect

In [59]:
# Helper visualization function
def draw_registration_result(source, target, transformation, text="None"):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp], text)

In [167]:
# グローバルレジストレーションの結果表示
def gr_result_info_print (gr_result, fitness=True, rmse=True, correspondence=True):
    if fitness:
        print(f"Fitness    :{gr_result.fitness:.8f}")
    if rmse:
        print(f"RMSE     :{gr_result.inlier_rmse:.8f}")
    if correspondence:
        print(f"Correspondence set:{len(gr_result.correspondence_set):}")

# グローバルレジストレーションの結果判定
def judgment_gr_result(gr_result, rmse = 0.10, fitness = 0.63):
    
    return gr_result.inlier_rmse >= rmse or gr_result.inlier_rmse == 0. or gr_result.fitness < fitness

In [155]:
# input用のplyファイルのリスト作成
def filelist_generator(distance, angle, hight, color, times, bunny=False, living=False):
        
    if bunny:
        # for sample file bunny
        # http://graphics.stanford.edu/data/3Dscanrep/
        # bunny.tar.gz
        enable_index = [0, 1, 2, 3, 4, 5, 6] # 0~6
        
        pcd_files = [
                fr"C:\work\NICT_AI\sample_ply\bunny\data\bun000.ply",
                fr"C:\work\NICT_AI\sample_ply\bunny\data\bun315.ply",
                fr"C:\work\NICT_AI\sample_ply\bunny\data\bun270.ply",
                fr"C:\work\NICT_AI\sample_ply\bunny\data\bun180.ply",
                fr"C:\work\NICT_AI\sample_ply\bunny\data\bun090.ply",
                fr"C:\work\NICT_AI\sample_ply\bunny\data\bun045.ply",            
            ]
        
        
        return pcd_files

    
    elif living:
    # for sample file living
    # http://redwood-data.org/indoor/data/
    # livingroom1-fragments-ply.zip
        enable_index = [0, 1, 2, 3, 4, 5, 6] # 0~56
        pcd_files = []
        for index in enable_index:
            pcd_files.append(fr"C:\work\NICT_AI\sample_ply\livingroom1-fragments-ply\cloud_bin_{index}.ply")

        return pcd_files
        
        
    else:
        # TODO: ファイルパス取得が煩雑すぎるので、正規表現でサーチできるようにする
        if (distance==2 and angle==15 and hight==1 and color=="b"):
            pcd_files = [[
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1\cam2\b\20201202155248\pcd_mask_mergeid_0000.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1\cam1\b\20201202155246\pcd_mask_mergeid_0000.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1\cam0\b\20201202155214\pcd_mask_mergeid_0000.ply",
                    ], [
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1\cam2\b\20201202155218\pcd_mask_mergeid_0001.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1\cam1\b\20201202155216\pcd_mask_mergeid_0001.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1\cam0\b\20201202155244\pcd_mask_mergeid_0001.ply",
            ]]
            pcd_files = pcd_files[times - 1]
        
        elif (distance==2 and angle==15 and hight==1 and color=="w"):
            pcd_files = [[
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1\cam2\w\20201202155347\pcd_mask_mergeid_0002.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1\cam1\w\20201202155345\pcd_mask_mergeid_0002.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1\cam0\w\20201202155343\pcd_mask_mergeid_0002.ply",
                    ], [
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1\cam2\w\20201202155319\pcd_mask_mergeid_0003.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1\cam1\w\20201202155317\pcd_mask_mergeid_0003.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1\cam0\w\20201202155315\pcd_mask_mergeid_0003.ply",
            ]]
            pcd_files = pcd_files[times - 1]
        
        elif (distance==2 and angle==15 and hight==1.5 and color=="b"):
            pcd_files = [[
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1.5\cam2\b\20201202170100\pcd_mask_mergeid_0000.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1.5\cam1\b\20201202170036\pcd_mask_mergeid_0000.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1.5\cam0\b\20201202170034\pcd_mask_mergeid_0000.ply",
                    ], [
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1.5\cam2\b\20201202170038\pcd_mask_mergeid_0001.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1.5\cam1\b\20201202170058\pcd_mask_mergeid_0001.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1.5\cam0\b\20201202170057\pcd_mask_mergeid_0001.ply",
            ]]
            pcd_files = pcd_files[times - 1]

        elif (distance==2 and angle==15 and hight==1.5 and color=="w"):
            pcd_files = [[
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1.5\cam2\w\20201202170127\pcd_mask_mergeid_0002.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1.5\cam1\w\20201202170125\pcd_mask_mergeid_0002.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1.5\cam0\w\20201202170123\pcd_mask_mergeid_0002.ply",
                    ], [
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1.5\cam2\w\20201202170150\pcd_mask_mergeid_0003.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1.5\cam1\w\20201202170148\pcd_mask_mergeid_0003.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a15_h1.5\cam0\w\20201202170146\pcd_mask_mergeid_0003.ply",
            ]]
            pcd_files = pcd_files[times - 1]

        elif (distance==2 and angle==30 and hight==1 and color=="b"):
            pcd_files = [[
                    fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1\cam2\b\20201202152623\pcd_mask_mergeid_0000.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1\cam1\b\20201202152621\pcd_mask_mergeid_0000.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1\cam0\b\20201202152544\pcd_mask_mergeid_0000.ply",
                    ], [
                    fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1\cam2\b\20201202152550\pcd_mask_mergeid_0001.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1\cam1\b\20201202152548\pcd_mask_mergeid_0001.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1\cam0\b\20201202152619\pcd_mask_mergeid_0001.ply",
            ]]
            pcd_files = pcd_files[times - 1]


        elif (distance==2 and angle==30 and hight==1 and color=="w"):
            pcd_files = [[
                    fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1\cam2\w\20201202152758\pcd_mask_mergeid_0002.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1\cam1\w\20201202152756\pcd_mask_mergeid_0002.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1\cam0\w\20201202152754\pcd_mask_mergeid_0002.ply",
                    ], [
                    fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1\cam2\w\20201202152701\pcd_mask_mergeid_0003.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1\cam1\w\20201202152659\pcd_mask_mergeid_0003.ply",
                    fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1\cam0\w\20201202152657\pcd_mask_mergeid_0003.ply",
            ]]
            pcd_files = pcd_files[times - 1]

        elif (distance==2 and angle==30 and hight==1.5 and color=="b"):
            pcd_files = [[
                fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1.5\cam2\b\20201202170510\pcd_mask_mergeid_0000.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1.5\cam1\b\20201202170527\pcd_mask_mergeid_0000.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1.5\cam0\b\20201202170526\pcd_mask_mergeid_0000.ply",
                ], [
                fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1.5\cam2\b\20201202170529\pcd_mask_mergeid_0001.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1.5\cam1\b\20201202170508\pcd_mask_mergeid_0001.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1.5\cam0\b\20201202170506\pcd_mask_mergeid_0001.ply",
            ]]
            pcd_files = pcd_files[times - 1]
            
        elif (distance==2 and angle==30 and hight==1.5 and color=="w"):
            pcd_files = [[
                fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1.5\cam2\w\20201202170556\pcd_mask_mergeid_0002.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1.5\cam1\w\20201202170616\pcd_mask_mergeid_0002.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1.5\cam0\w\20201202170553\pcd_mask_mergeid_0002.ply",
                ], [
                fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1.5\cam2\w\20201202170618\pcd_mask_mergeid_0003.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1.5\cam1\w\20201202170554\pcd_mask_mergeid_0003.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a30_h1.5\cam0\w\20201202170615\pcd_mask_mergeid_0003.ply",
            ]]
            pcd_files = pcd_files[times - 1]

        elif (distance==2 and angle==45 and hight==1 and color=="b"):
            pcd_files = [[
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1\cam2\b\20201202160205\pcd_mask_mergeid_0000.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1\cam1\b\20201202160226\pcd_mask_mergeid_0000.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1\cam0\b\20201202160225\pcd_mask_mergeid_0000.ply",
                ], [
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1\cam2\b\20201202160228\pcd_mask_mergeid_0001.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1\cam1\b\20201202160203\pcd_mask_mergeid_0001.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1\cam0\b\20201202160159\pcd_mask_mergeid_0001.ply",
            ]]
            pcd_files = pcd_files[times - 1]
    
        elif (distance==2 and angle==45 and hight==1 and color=="w"):
            pcd_files = [[
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1\cam2\w\20201202160302\pcd_mask_mergeid_0002.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1\cam1\w\20201202160323\pcd_mask_mergeid_0002.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1\cam0\w\20201202160321\pcd_mask_mergeid_0002.ply",
                ], [
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1\cam2\w\20201202160325\pcd_mask_mergeid_0003.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1\cam1\w\20201202160300\pcd_mask_mergeid_0003.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1\cam0\w\20201202160258\pcd_mask_mergeid_0003.ply",
            ]]
            pcd_files = pcd_files[times - 1]
    
        elif (distance==2 and angle==45 and hight==1.5 and color=="b"):
            pcd_files = [[
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1.5\cam2\b\20201202170904\pcd_mask_mergeid_0000.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1.5\cam1\b\20201202170843\pcd_mask_mergeid_0000.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1.5\cam0\b\20201202170842\pcd_mask_mergeid_0000.ply",
                ], [
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1.5\cam2\b\20201202170845\pcd_mask_mergeid_0001.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1.5\cam1\b\20201202170902\pcd_mask_mergeid_0001.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1.5\cam0\b\20201202170859\pcd_mask_mergeid_0001.ply",
            ]]
            pcd_files = pcd_files[times - 1]

        elif (distance==2 and angle==45 and hight==1.5 and color=="w"):
            pcd_files = [[
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1.5\cam2\w\20201202171001\pcd_mask_mergeid_0002.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1.5\cam1\w\20201202170959\pcd_mask_mergeid_0002.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1.5\cam0\w\20201202170957\pcd_mask_mergeid_0002.ply",
                ], [
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1.5\cam2\w\20201202170935\pcd_mask_mergeid_0003.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1.5\cam1\w\20201202170934\pcd_mask_mergeid_0003.ply",
                fr"C:\work\NICT_AI\narrow_angle\d2_a45_h1.5\cam0\w\20201202170932\pcd_mask_mergeid_0003.ply",
            ]]
            pcd_files = pcd_files[times - 1]

            
        elif (distance==4 and angle==15 and hight==1 and color=="b"):
            pcd_files = [[
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1\cam2\b\20201202160908\pcd_mask_mergeid_0000.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1\cam1\b\20201202160906\pcd_mask_mergeid_0000.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1\cam0\b\20201202160904\pcd_mask_mergeid_0000.ply",
                ], [
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1\cam2\b\20201202160937\pcd_mask_mergeid_0001.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1\cam1\b\20201202160936\pcd_mask_mergeid_0001.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1\cam0\b\20201202160933\pcd_mask_mergeid_0001.ply",
            ]]
            pcd_files = pcd_files[times - 1]

        elif (distance==4 and angle==15 and hight==1 and color=="w"):
            pcd_files = [[
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1\cam2\w\20201202161028\pcd_mask_mergeid_0002.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1\cam1\w\20201202161026\pcd_mask_mergeid_0002.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1\cam0\w\20201202161024\pcd_mask_mergeid_0002.ply",
                ], [
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1\cam2\w\20201202161052\pcd_mask_mergeid_0003.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1\cam1\w\20201202161051\pcd_mask_mergeid_0003.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1\cam0\w\20201202161049\pcd_mask_mergeid_0003.ply",
            ]]
            pcd_files = pcd_files[times - 1]

        elif (distance==4 and angle==15 and hight==1.5 and color=="b"):
            pcd_files = [[
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1.5\cam2\b\20201202165313\pcd_mask_mergeid_0000.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1.5\cam1\b\20201202165312\pcd_mask_mergeid_0000.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1.5\cam0\b\20201202165310\pcd_mask_mergeid_0000.ply",
                ], [
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1.5\cam2\b\20201202165333\pcd_mask_mergeid_0001.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1.5\cam1\b\20201202165331\pcd_mask_mergeid_0001.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1.5\cam0\b\20201202165330\pcd_mask_mergeid_0001.ply",
            ]]
            pcd_files = pcd_files[times - 1]
            
        elif (distance==4 and angle==15 and hight==1.5 and color=="w"):
            pcd_files = [[
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1.5\cam2\w\20201202165401\pcd_mask_mergeid_0002.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1.5\cam1\w\20201202165359\pcd_mask_mergeid_0002.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1.5\cam0\w\20201202165357\pcd_mask_mergeid_0002.ply",
                ], [
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1.5\cam2\w\20201202165424\pcd_mask_mergeid_0003.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1.5\cam1\w\20201202165422\pcd_mask_mergeid_0003.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a15_h1.5\cam0\w\20201202165420\pcd_mask_mergeid_0003.ply",
            ]]
            pcd_files = pcd_files[times - 1]

        elif (distance==4 and angle==30 and hight==1 and color=="b"):
            pcd_files = [[
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1\cam2\b\20201202161701\pcd_mask_mergeid_0000.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1\cam1\b\20201202161659\pcd_mask_mergeid_0000.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1\cam0\b\20201202161657\pcd_mask_mergeid_0000.ply",
                ], [
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1\cam2\b\20201202161739\pcd_mask_mergeid_0001.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1\cam1\b\20201202161737\pcd_mask_mergeid_0001.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1\cam0\b\20201202161735\pcd_mask_mergeid_0001.ply",
            ]]
            pcd_files = pcd_files[times - 1]
    
        elif (distance==4 and angle==30 and hight==1 and color=="w"):
            pcd_files = [[
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1\cam2\w\20201202161811\pcd_mask_mergeid_0002.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1\cam1\w\20201202161809\pcd_mask_mergeid_0002.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1\cam0\w\20201202161807\pcd_mask_mergeid_0002.ply",
                ], [
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1\cam2\w\20201202161841\pcd_mask_mergeid_0003.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1\cam1\w\20201202161839\pcd_mask_mergeid_0003.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1\cam0\w\20201202161835\pcd_mask_mergeid_0003.ply",
            ]]
            pcd_files = pcd_files[times - 1]

        elif (distance==4 and angle==30 and hight==1.5 and color=="b"):
            pcd_files = [[
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1.5\cam2\b\20201202164910\pcd_mask_mergeid_0000.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1.5\cam1\b\20201202164908\pcd_mask_mergeid_0000.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1.5\cam0\b\20201202164907\pcd_mask_mergeid_0000.ply",
                ], [
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1.5\cam2\b\20201202164936\pcd_mask_mergeid_0001.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1.5\cam1\b\20201202164934\pcd_mask_mergeid_0001.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1.5\cam0\b\20201202164932\pcd_mask_mergeid_0001.ply",
            ]]
            pcd_files = pcd_files[times - 1]

        elif (distance==4 and angle==30 and hight==1.5 and color=="w"):
            pcd_files = [[
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1.5\cam2\w\20201202165001\pcd_mask_mergeid_0002.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1.5\cam1\w\20201202164959\pcd_mask_mergeid_0002.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1.5\cam0\w\20201202164957\pcd_mask_mergeid_0002.ply",
                ], [
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1.5\cam2\w\20201202165030\pcd_mask_mergeid_0003.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1.5\cam1\w\20201202165028\pcd_mask_mergeid_0003.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a30_h1.5\cam0\w\20201202165026\pcd_mask_mergeid_0003.ply",
            ]]
            pcd_files = pcd_files[times - 1]

        elif (distance==4 and angle==45 and hight==1 and color=="b"):
            pcd_files = [[
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1\cam2\b\20201202162440\pcd_mask_mergeid_0000.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1\cam1\b\20201202162438\pcd_mask_mergeid_0000.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1\cam0\b\20201202162436\pcd_mask_mergeid_0000.ply",
                ], [
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1\cam2\b\20201202162512\pcd_mask_mergeid_0001.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1\cam1\b\20201202162510\pcd_mask_mergeid_0001.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1\cam0\b\20201202162508\pcd_mask_mergeid_0001.ply",
            ]]
            pcd_files = pcd_files[times - 1]

        elif (distance==4 and angle==45 and hight==1 and color=="w"):
            pcd_files = [[
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1\cam2\w\20201202162545\pcd_mask_mergeid_0002.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1\cam1\w\20201202162543\pcd_mask_mergeid_0002.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1\cam0\w\20201202162541\pcd_mask_mergeid_0002.ply",
                ], [
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1\cam2\w\20201202162609\pcd_mask_mergeid_0003.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1\cam1\w\20201202162607\pcd_mask_mergeid_0003.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1\cam0\w\20201202162605\pcd_mask_mergeid_0003.ply",
            ]]
            pcd_files = pcd_files[times - 1]

        elif (distance==4 and angle==45 and hight==1.5 and color=="b"):
            pcd_files = [[
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1.5\cam2\b\20201202164309\pcd_mask_mergeid_0000.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1.5\cam1\b\20201202164307\pcd_mask_mergeid_0000.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1.5\cam0\b\20201202164305\pcd_mask_mergeid_0000.ply",
                ], [
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1.5\cam2\b\20201202164333\pcd_mask_mergeid_0001.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1.5\cam1\b\20201202164332\pcd_mask_mergeid_0001.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1.5\cam0\b\20201202164330\pcd_mask_mergeid_0001.ply",
            ]]
            pcd_files = pcd_files[times - 1]

        elif (distance==4 and angle==45 and hight==1.5 and color=="w"):
            pcd_files = [[
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1.5\cam2\w\20201202164439\pcd_mask_mergeid_0002.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1.5\cam1\w\20201202164438\pcd_mask_mergeid_0002.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1.5\cam0\w\20201202164436\pcd_mask_mergeid_0002.ply",
                ], [
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1.5\cam2\w\20201202164501\pcd_mask_mergeid_0003.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1.5\cam1\w\20201202164500\pcd_mask_mergeid_0003.ply",
                fr"C:\work\NICT_AI\narrow_angle\d4_a45_h1.5\cam0\w\20201202164458\pcd_mask_mergeid_0003.ply",
            ]]
            pcd_files = pcd_files[times - 1]
            
        else :
            raise Exception(f"Condition does not match")
          

        return pcd_files

In [61]:
def add_color_normal(pcd): # in-place coloring and adding normal
    pcd.paint_uniform_color(np.random.rand(3))
    size = np.abs((pcd.get_max_bound() - pcd.get_min_bound())).max() / 30
    kdt_n = o3d.geometry.KDTreeSearchParamHybrid(radius=size, max_nn=50)
    pcd.estimate_normals(kdt_n)

In [62]:
def merge_pcd(pcds):
    # 複数の点群を1つの点群にマージする

    all_points = []
    for pcd in pcds:
        all_points.append(np.asarray(pcd.points))

    merged_pcd = o3d.geometry.PointCloud()
    merged_pcd.points = o3d.utility.Vector3dVector(np.vstack(all_points))

    return merged_pcd

In [63]:
def preprocess_point_cloud(pcd, voxel_size):

    # ダウンサンプリング
    pcd_down = pcd.voxel_down_sample(voxel_size)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size, max_nn=50))

    # 特徴量計算
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 10, max_nn=50))
    return pcd_down, pcd_fpfh

In [64]:
def prepare_dataset(voxel_size, source, target):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    
    source_down, source_fpfh = preprocess_point_cloud(source_temp, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target_temp, voxel_size)

    return source_down, target_down, source_fpfh, target_fpfh

In [65]:
def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):

    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        max_correspondence_distance=voxel_size * 2,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        ransac_n=4,
        checkers=[
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(voxel_size * 2)
            ], 
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(max_iteration=400000, max_validation=500))#400000, 500
    return result

In [66]:
def execute_fast_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):

    result = o3d.pipelines.registration.registration_fast_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.pipelines.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=voxel_size * 2))
    return result

In [67]:
def p2p_ICP_register(source, target, threshold, trans_init, iteration):
    
    print("\nApply point-to-point ICP")
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(
            relative_fitness = 1e-5, # fitnessの変化分がこれより小さくなったら収束
            relative_rmse = 1e-5, # RMSEの変化分がこれより小さくなったら収束
            max_iteration = iteration)) # 反復回数

    gr_result_info_print(reg_p2p)
    draw_registration_result(source, target, reg_p2p.transformation, "point-to-point ICP")
    
    return reg_p2p

In [136]:
def p2l_ICP_register(source, target, threshold, trans_init, iteration):

    print("\nApply point-to-plane ICP")
    loss = o3d.pipelines.registration.TukeyLoss(k=0.1)
    reg_p2l = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPlane(loss),
        o3d.pipelines.registration.ICPConvergenceCriteria(
            relative_fitness = 1e-5, # fitnessの変化分がこれより小さくなったら収束
            relative_rmse = 1e-5, # RMSEの変化分がこれより小さくなったら収束
            max_iteration = iteration)) # 反復回数

    gr_result_info_print(reg_p2l)
#     draw_registration_result(source, target, reg_p2l.transformation, "point-to-plane ICP")
    
    return reg_p2l

In [104]:
#点群ノイズの除外処理http://whitewell.sakura.ne.jp/Open3D/PointCloudOutlierRemoval.html
def display_inlier_outlier(cloud, ind):
    # inlier_cloud = o3d.geometry.PointCloud.select_down_sample(ind)
    # outlier_cloud = o3d.geometry.PointCloud.select_down_sample(cloud,ind, invert=True)

    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    #  print("Showing outliers (red) and inliers (gray): ")
    # outlier_cloud.paint_uniform_color([1, 0, 0])
    # inlier_cloud.paint_uniform_color([0, 0, 1])
    # o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])
    # o3d.visualization.draw_geometries([inlier_cloud])

    return inlier_cloud,outlier_cloud

def remove_noise_ply(pcd):
    # (1) Load a ply point cloud, print it, and render it
    pcd_temp = copy.deepcopy(pcd)

    # (2) Downsample the point cloud with a voxel
    voxel_down_pcd = pcd.voxel_down_sample(voxel_size=0.02) # default:0.02

    # (3) Every 5th points are selected
    uni_down_pcd = pcd.uniform_down_sample(every_k_points=5)# default:5

    # (4) Statistical oulier removal
    cl, ind = voxel_down_pcd.remove_statistical_outlier(nb_neighbors=20, # default: nb_neighbors:20, std_ratio:2.0
                                                        std_ratio=2.0)
    display_inlier_outlier(voxel_down_pcd, ind)

    # (5) Radius oulier removal
    cl, ind = voxel_down_pcd.remove_radius_outlier(nb_points=16, radius=0.05) # default: nb_points:16, radius:0.05
    inlier_cloud,outlier_cloud=display_inlier_outlier(voxel_down_pcd, ind)

    return inlier_cloud

In [95]:
def register(source, target, size):

    # TO DO:グローバルレジストレーションの分岐削除を検討・反映する 
    # voxel size設定 model点群の1/10を基本サイズにする
#     size = np.abs((source.get_max_bound() - source.get_min_bound())).max() / 10
#     size_list = [size/i for i in range(1,11)]
    
#     voxel_size_index = 2     #  1/10 * (n+1)
#     voxel_size = size_list[voxel_size_index]
#     print(f"use voxel size:{size_list[voxel_size_index]:}")
    voxel_size = size

    # 法線推定
    kdt_n = o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size, max_nn=50)
    source.estimate_normals(kdt_n)
    target.estimate_normals(kdt_n)
    
    # GR(グローバルレジストレーション)準備
    use_rgr_fgr = {"ransac":True, "fast":False}
    gr_result={}
    
    source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size, source, target)
#     print("\nSearching for best transformation...")
    
    # RANSAC GR
    if use_rgr_fgr["ransac"]:
        gr_ransac_result = execute_global_registration(source_down, target_down,
                                        source_fpfh, target_fpfh, voxel_size)
        gr_result["ransac"] = gr_ransac_result
#         print("ransac 1st result")
#         gr_result_info_print(gr_ransac_result)
#         draw_registration_result(source, target, gr_ransac_result.transformation, "RGR_init")

    # Fast GR
    if use_rgr_fgr["fast"]:
        gr_fast_result = execute_fast_global_registration(source_down, target_down,
                                        source_fpfh, target_fpfh, voxel_size)
        gr_result["fast"] = gr_ransac_result
#         print("fast 1st result")
#         gr_result_info_print(gr_fast_result)
#         draw_registration_result(source, target, gr_fast_result.transformation, "FGR_init")

    if not (use_rgr_fgr["ransac"] or use_rgr_fgr["fast"]):
        raise Exception("All GR flag is False!")
    
    for key in gr_result.keys():
        # 有効な結果を算出するまでGRループ 2000回で脱出
        cnt = 0
        while(judgment_gr_result(gr_result[key])):
            cnt+=1
#             print(key, " Loop: ", cnt)
            gr_result[key] = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh, voxel_size)
#             gr_result_info_print(gr_result[key])
#             print("")
            if cnt == 2000:
                print("Cannot search for best transformation.")
                # TODO: 2000回探しても見つからない場合に条件を緩和するロジック要検討 このときにvoxcel size:1/10～100で変動させてみるのもあり
                break
#         print(key, " Finished loop : ", cnt)
#         gr_result_info_print(gr_result[key])
#         draw_registration_result(source, target, gr_result[key].transformation, f"{key}GR_{cnt}")
    
        # ICP
        # http://www.open3d.org/docs/latest/tutorial/pipelines/icp_registration.html
        # http://whitewell.sakura.ne.jp/Open3D_2018/
        iteration = 20000 # 20000 10
        # point-to-point ICP
#         p2p_ICP_register(source, target, voxel_size, gr_result[key].transformation, iteration)

        # point-to-plane ICP
        icp_result = p2l_ICP_register(source, target, voxel_size, gr_result[key].transformation, iteration)

    return icp_result

In [164]:
if __name__ == "__main__":
    print("Open3D version:", o3d.__version__)
    
    # ファイルパラメータ
    distance = 4
    angle = 45
    hight = 1.5
    color ="w"
    times = 2
    bunny=False
    living=False
    pcd_files=  filelist_generator(distance, angle, hight, color, times, bunny, living)
    
    pcds = []
    all_points = []
    all_colors = []

    for f in pcd_files:
        if not os.path.exists(f):
            raise Exception(f"File does not exist!\n{f}")
        pcd = o3d.io.read_point_cloud(f)
        print("Read point cloud file\n ", f, "\n ", pcd)
        
        # ノイズ除去
        pcd_tem = remove_noise_ply(pcd)
        cnt_org = np.array(pcd.points).shape[0]
        cnt_noise = np.array(pcd_tem.points).shape[0]
        if cnt_noise / cnt_org > 0.05:
            pcd=pcd_tem
            print(f" Enable remove noize :{np.array(pcd_tem.points).shape[0]:}")
        
        all_points.append(np.asarray(pcd.points))
        all_colors.append(np.asarray(pcd.colors))
#         add_color_normal(pcd)
        pcds.append(pcd)
    # 初期点群描画
#     o3d.visualization.draw_geometries(pcds, "input pcds")

    size = np.abs((pcds[0].get_max_bound() - pcds[0].get_min_bound())).max() / 30
    # TODO: 先頭ファイルの点群サイズでvoxel sizeを作成している 合成時ごとにする必要がないか要検討
    
    pose_graph = o3d.pipelines.registration.PoseGraph()
    accum_pose = np.identity(4) # id0から各ノードへの累積姿勢
    pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(accum_pose))

    n_pcds = len(pcds)
    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            source = pcds[source_id]
            target = pcds[target_id]
            
            trans = register(source, target, size)
            GTG_mat = \
                o3d.pipelines.registration.get_information_matrix_from_point_clouds(source, target, size, trans.transformation)
            # これが点の情報を含む

            if target_id == source_id + 1: # 次のidの点群ならaccum_poseにposeを積算
                accum_pose = trans.transformation @ accum_pose
                pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(np.linalg.inv(accum_pose)))
                # 各ノードは，このノードのidからid0への変換姿勢を持つので，invする
                # そうでないならnodeは作らない
            pose_graph.edges.append(o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                       target_id,
                                                       trans.transformation,
                                                       GTG_mat,
                                                       uncertain=True)) # bunnyの場合，隣でも怪しいので全部True
            
    # 設定
    solver = o3d.pipelines.registration.GlobalOptimizationLevenbergMarquardt()
    criteria = o3d.pipelines.registration.GlobalOptimizationConvergenceCriteria()
    option = o3d.pipelines.registration.GlobalOptimizationOption(
             max_correspondence_distance=size / 10,
             edge_prune_threshold=size / 10,
             reference_node=0)

    # 最適化
    o3d.pipelines.registration.global_optimization(pose_graph,
                            method=solver,
                            criteria=criteria,
                            option=option)

    # 推定した姿勢で点群を変換
    for pcd_id in range(n_pcds):
        trans = pose_graph.nodes[pcd_id].pose
        pcds[pcd_id].transform(trans)
    
#     o3d.visualization.draw_geometries(pcds, "aligned")
    
    pcd_merge = o3d.geometry.PointCloud()
    pcd_merge.points = o3d.utility.Vector3dVector(np.vstack(all_points))
    pcd_merge.colors = o3d.utility.Vector3dVector(np.vstack(all_colors))    
    o3d.visualization.draw_geometries([pcd_merge], "merged")
    
    # 合成点群保存
    if bunny:
        file_name_temp = f"result_ply/merge_bunny.ply"
    elif living:
        file_name_temp = f"result_ply/merge_living.ply"
    else :
        file_name_temp = f"result_ply/merge_d{distance}_a{angle}_h{hight}_{color}_{times}.ply"
    o3d.io.write_point_cloud(file_name_temp, pcd_merge)

    
    print("\n\nFinished.")

Open3D version: 0.11.2
Read point cloud file
  C:\work\NICT_AI\narrow_angle\d4_a45_h1.5\cam2\w\20201202164501\pcd_mask_mergeid_0003.ply 
  PointCloud with 43232 points.
 Enable remove noize :2771
Read point cloud file
  C:\work\NICT_AI\narrow_angle\d4_a45_h1.5\cam1\w\20201202164500\pcd_mask_mergeid_0003.ply 
  PointCloud with 69313 points.
 Enable remove noize :4813
Read point cloud file
  C:\work\NICT_AI\narrow_angle\d4_a45_h1.5\cam0\w\20201202164458\pcd_mask_mergeid_0003.ply 
  PointCloud with 56898 points.
 Enable remove noize :3291

Apply point-to-plane ICP
Fitness    :0.80512450
RMSE     :0.02990285
Correspondence set:2231

Apply point-to-plane ICP
Fitness    :0.57416095
RMSE     :0.02818114
Correspondence set:1591

Apply point-to-plane ICP
Fitness    :0.43050073
RMSE     :0.02915604
Correspondence set:2072
[Open3D WARNING] Certain-edge subset of PoseGraph is not connected.


Finished.


In [166]:
# ファイルサーチ試し
import glob
import os

# ファイルパラメータ
distance=2
angle=15
hight=1.5
color="b"
times=0

# ファイルパステンプレート
file_temp = \
    fr"C:\work\NICT_AI\narrow_angle\d{distance}_a{angle}_h{hight}\cam[0-2]\{color}\*\pcd_mask_mergeid_[0-9][0-9]*.ply"

# パラメータからファイル検索、2パターンのファイルパスまで絞る
file_list = [file_path for file_path in glob.glob(file_temp)]

# 最後のID部を絞り込み、対象の3ファイルに限定
list_a = []
for name in file_list:
    num = os.path.splitext(os.path.basename(name))[0].split("_")[3]
    list_a.append(num)

file_list2 = [name for name  in file_list if os.path.splitext(os.path.basename(name))[0].split("_")[3] == list(set(list_a))[0]]
for file_path in file_list2:
    print(file_path)


C:\work\NICT_AI\narrow_angle\d2_a15_h1.5\cam0\b\20201202170034\pcd_mask_mergeid_0000.ply
C:\work\NICT_AI\narrow_angle\d2_a15_h1.5\cam1\b\20201202170036\pcd_mask_mergeid_0000.ply
C:\work\NICT_AI\narrow_angle\d2_a15_h1.5\cam2\b\20201202170100\pcd_mask_mergeid_0000.ply
